In [30]:
import random
import json
import numpy as np
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [57]:
def label (dataCategory,datas):
    labels = []
    if dataCategory == 'train':
        for idx in range(len(datas)):
            data = datas[idx]    
            author = np.array(data['authors'])
            prolific_author = author<100
            prolific_author = author[prolific_author]
            empty_author = np.zeros(101) # 100 plus 1 negative
            if len(prolific_author)!=0:
                empty_author[prolific_author]=1
            else:
                empty_author[100]=1
            labels.append(empty_author)
 
    return labels

In [67]:
def text_feature (dataCategory, datas, model):
    title = []
    abstract = []
    for data in datas:
        single_abstract = data['abstract']
        single_title = data['title']
        title.append(single_title)
        abstract.append(single_abstract)
    common_texts = title+abstract
    # doc2vec code from https://radimrehurek.com/gensim/models/doc2vec.html
    documents = [TaggedDocument(doc,[i])for i,doc in enumerate(common_texts)]
    if dataCategory == 'train':
        model=Doc2Vec(documents,vector_size = 256,window = 2,min_count = 1)#no worker
        model.train(documents, total_examples=model.corpus_count, epochs=30)# 20
    full_text = []
    for data in datas:
        abstract2 = data['abstract']
        title2 = data['title']
        sums = abstract2+title2
        text_empty = np.zeros(256)
        for word in sums:
            word_single = []
            word_single.append(str(word))
        text_empty+= model.infer_vector(word_single)   
        full_text.append(text_empty)
    full_text = np.array(full_text)
    return (full_text, model)

In [40]:
def onehot_feature (dataCategory,datas, co_idx={}):
    if dataCategory == 'train':
        author = {}
        co_idx = {}
        coauthor = []
        for record in datas:
            author_record = np.array(record['authors'])
            pa_bol = author_record<100
            ca_bol = author_record>=100
            co_author = author_record[ca_bol]
            if author_record[pa_bol == True].size != 0:
                for i in co_author:
                    author[i] = 1
        index = author.keys()
        coauthorList = list(index)
        
        for i in range(len(coauthorList)):
            co_idx[coauthorList[i]]=i

        for record in datas:
            author_record = np.array(record['authors'])
            pa_bol = author_record<100
            ca_bol = author_record>=100
            co_author = author_record[ca_bol]
            colen = len(coauthorList)
            co_empty = np.zeros(colen+1)
            if author_record[pa_bol == True].size == 0:
                co_empty[colen]=1.
            else:
                for j in co_author:
                    idx = co_idx[j]
                    co_empty[idx] = 1
            coauthor.append(co_empty)
        
        coauthor = np.array(coauthor)
        
        # year 
        year = []
        for record in datas:
            year_empty = np.zeros(20)
            year_record = record['year']
            year_empty[year_record] = 1
            year.append(year_empty)
        year = np.array(year)
        
        # venue    
        venue = []
        np.array(venue)
        for record in datas:
            venue_empty = np.zeros(466)# 465 +1 empty
            venue_data = record['venue']
            if venue_data == '':
                venue_data = 465
            venue_empty[venue_data] = 1
            venue.append( venue_empty)
        onehot = np.concatenate((venue,coauthor),axis =1 )
        return (onehot, co_idx)
    
    elif dataCategory == 'test':
        author = {}
        index = co_idx.keys()
        coauthorList = list(index)
        coauthor = []
        for record in datas:
            author_record = np.array(record['coauthors'])
            colen = len(coauthorList)
            co_empty = np.zeros(colen+1)
            for au in author_record:
                if au in coauthorList:
                    co_empty[co_idx[au]] = 1
                else:
                    co_empty[-1] = 1
            coauthor.append(co_empty)
        coauthor = np.array(coauthor)
        
        # year 
        year = []
        for record in datas:
            year_empty = np.zeros(20)
            year_record = record['year']
            year_empty[year_record] = 1
            year.append(year_empty)
        year = np.array(year)
        
        #venue
        venue = []
        np.array(venue)
        for record in datas:
            venue_empty = np.zeros(466)# 465 +1 empty
            venue_data = record['venue']
            if venue_data == '':
                venue_data = 465
            venue_empty[venue_data] = 1
            venue.append( venue_empty)
        onehot = np.concatenate((venue,coauthor),axis =1 )
        return (onehot, None)

In [59]:
def feature (dataCategory,datas, model, co_list):
    text, model = text_feature (dataCategory, datas, model)
    one_hot, co_list = onehot_feature(dataCategory,datas, co_list)
    feature = np.concatenate((text,one_hot),axis =1 )
    return (feature, model, co_list)
    

In [42]:
train_data = json.load(open('train.json'))
test_data = json.load(open('test.json'))

In [68]:
model = None
co_idx = None
train_feature, model, co_idx = feature('train',train_data, model, co_idx)
np.save('feature_train.npy',train_feature)

In [44]:
test_feature, _, _ = feature('test',test_data, model, co_idx)
np.save('feature_test.npy',test_feature)

In [55]:
label_train = label ('train',train_data)
np.save('labels.npy',label_train)

In [49]:
test = []
for f in test_feature:
    test.append(np.where(f[-6576:] == 1))
    